<a href="https://www.kaggle.com/code/jobaerislam/lung-colon?scriptVersionId=238431785" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.applications import MobileNetV2
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

2025-05-07 19:18:51.714613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746645531.737412     517 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746645531.744329     517 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
dataset_dir = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set'

In [3]:
img_height, img_width = 224, 224

In [4]:
def load_data(dataset_dir):
    images = []
    labels = []

    # List the main folders
    main_folders = ['colon_image_sets', 'lung_image_sets']

    for main_folder in main_folders:
        main_path = os.path.join(dataset_dir, main_folder)
        for folder in os.listdir(main_path):
            label = folder
            folder_path = os.path.join(main_path, folder)
            for filename in os.listdir(folder_path):
                img = cv2.imread(os.path.join(folder_path, filename))
                img = cv2.resize(img, (img_height, img_width))
                images.append(img)
                labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    return images, labels

images, labels = load_data(dataset_dir)

In [5]:
label_dict = {
    'colon_aca': 0, 
    'colon_n': 1, 
    'lung_aca': 2, 
    'lung_n': 3, 
    'lung_scc': 4
}
labels = np.array([label_dict[label] for label in labels])
labels = to_categorical(labels)

In [6]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, labels, test_size=0.3, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [7]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

I0000 00:00:1746645641.154510     517 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(5, activation='softmax')
])

In [9]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [10]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=50,
          validation_data=(X_val, y_val),
          callbacks=[early_stop])

Epoch 1/50


I0000 00:00:1746645685.022525     551 service.cc:148] XLA service 0x7dbea80ca990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746645685.022561     551 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1746645688.167802     551 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1746645693.140545     551 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746645693.338933     551 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


  1/547 ━━━━━━━━━━━━━━━━━━━━ 8:21:22 55s/step - accuracy: 0.3125 - loss: 1.8751

I0000 00:00:1746645704.020475     551 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


546/547 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9167 - loss: 0.2585

E0000 00:00:1746645751.498600     550 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1746645751.700543     550 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


547/547 ━━━━━━━━━━━━━━━━━━━━ 124s 126ms/step - accuracy: 0.9168 - loss: 0.2582 - val_accuracy: 0.3672 - val_loss: 26.8101
Epoch 2/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.9733 - loss: 0.0824 - val_accuracy: 0.3093 - val_loss: 34.4777
Epoch 3/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 76ms/step - accuracy: 0.9775 - loss: 0.0683 - val_accuracy: 0.3408 - val_loss: 16.7509
Epoch 4/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.9819 - loss: 0.0565 - val_accuracy: 0.2595 - val_loss: 11.5968
Epoch 5/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.9873 - loss: 0.0381 - val_accuracy: 0.4011 - val_loss: 12.9998
Epoch 6/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.9800 - loss: 0.0710 - val_accuracy: 0.3536 - val_loss: 35.1523
Epoch 7/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.9890 - loss: 0.0377 - val_accuracy: 0.2632 - val_loss: 21.1552
Epoch 8/50
547/547 ━━━━━━━━━━━━━━━━━━━━ 42s 76ms/step - accuracy: 0.9895 - loss: 0.0326 - va